## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_copy import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pangnirtung,CA,66.1451,-65.7125,33.80,64,75,3.44,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,40.66,56,75,12.66,broken clouds
2,2,Los Llanos De Aridane,ES,28.6585,-17.9182,73.15,77,75,6.91,volcanic ash
3,3,Barbacoas,CO,1.6715,-78.1398,80.83,83,71,1.72,broken clouds
4,4,Deputatskiy,RU,69.3000,139.9000,26.46,86,100,16.33,overcast clouds


In [3]:
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Los Llanos De Aridane,ES,28.6585,-17.9182,73.15,77,75,6.91,volcanic ash
3,3,Barbacoas,CO,1.6715,-78.1398,80.83,83,71,1.72,broken clouds
5,5,Castro,BR,-24.7911,-50.0119,72.36,54,12,3.42,few clouds
6,6,Rikitea,PF,-23.1203,-134.9692,73.78,73,15,14.94,few clouds
7,7,Jamestown,US,42.0970,-79.2353,62.51,72,1,5.01,clear sky
8,8,Kavieng,PG,-2.5744,150.7967,82.33,78,9,15.08,clear sky
9,9,Puerto Ayora,EC,-0.7393,-90.3518,71.56,87,99,8.57,overcast clouds
10,10,Anloga,GH,5.7947,0.8973,86.22,83,99,10.76,overcast clouds
14,14,Cape Town,ZA,-33.9258,18.4232,68.11,69,0,20.71,clear sky
17,17,Tezu,IN,27.9167,96.1667,78.49,82,31,4.56,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                408
City                   408
Country                406
Lat                    408
Lng                    408
Max Temp               408
Humidity               408
Cloudiness             408
Wind Speed             408
Current Description    408
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                406
City                   406
Country                406
Lat                    406
Lng                    406
Max Temp               406
Humidity               406
Cloudiness             406
Wind Speed             406
Current Description    406
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Los Llanos De Aridane,ES,73.15,volcanic ash,28.6585,-17.9182,
3,Barbacoas,CO,80.83,broken clouds,1.6715,-78.1398,
5,Castro,BR,72.36,few clouds,-24.7911,-50.0119,
6,Rikitea,PF,73.78,few clouds,-23.1203,-134.9692,
7,Jamestown,US,62.51,clear sky,42.0970,-79.2353,
8,Kavieng,PG,82.33,clear sky,-2.5744,150.7967,
9,Puerto Ayora,EC,71.56,overcast clouds,-0.7393,-90.3518,
10,Anloga,GH,86.22,overcast clouds,5.7947,0.8973,
14,Cape Town,ZA,68.11,clear sky,-33.9258,18.4232,
17,Tezu,IN,78.49,scattered clouds,27.9167,96.1667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
#no rows needed to be dropped

City                   406
Country                406
Max Temp               406
Current Description    406
Lat                    406
Lng                    406
Hotel Name             406
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "VacationPy_Database.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))